<a href="https://colab.research.google.com/github/mimansha1612/LoanDecisionProject/blob/main/Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

df = pd.read_csv('/content/accepted_2007_to_2018Q4.csv.gz', low_memory=False)
df.head()


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
cols = [
    'loan_amnt',
    'int_rate',
    'annual_inc',
    'dti',
    'fico_range_low',
    'fico_range_high',
    'purpose',
    'term',
    'loan_status'
]

df = df[cols]
df.head()


,loan_amnt,int_rate,annual_inc,dti,fico_range_low,fico_range_high,purpose,term,loan_status
0,3600.0,13.99,55000.0,5.91,675.0,679.0,debt_consolidation,36 months,Fully Paid
1,24700.0,11.99,65000.0,16.06,715.0,719.0,small_business,36 months,Fully Paid
2,20000.0,10.78,63000.0,10.78,695.0,699.0,home_improvement,60 months,Fully Paid
3,35000.0,14.85,110000.0,17.06,785.0,789.0,debt_consolidation,60 months,Current
4,10400.0,22.45,104433.0,25.37,695.0,699.0,major_purchase,60 months,Fully Paid


In [7]:
# Map loan_status into binary
df['loan_status'] = df['loan_status'].map({
    'Fully Paid': 0,
    'Charged Off': 1
})

# Remove rows where loan_status is not 0 or 1
df = df.dropna(subset=['loan_status'])
df['loan_status'].value_counts()


,count
loan_status,
0.0,1076751
1.0,268559


In [9]:
# Numeric columns
num_cols = ['loan_amnt','int_rate','annual_inc','dti','fico_range_low','fico_range_high']

# Fill numeric missing values
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# Categorical columns
cat_cols = ['purpose','term']

# Fill categorical missing values
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])


In [10]:
df = pd.get_dummies(df, columns=['purpose','term'], drop_first=True)
df.head()


,loan_amnt,int_rate,annual_inc,dti,fico_range_low,fico_range_high,loan_status,purpose_credit_card,purpose_debt_consolidation,purpose_educational,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 60 months
0,3600.0,13.99,55000.0,5.91,675.0,679.0,0.0,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,24700.0,11.99,65000.0,16.06,715.0,719.0,0.0,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,20000.0,10.78,63000.0,10.78,695.0,699.0,0.0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,10400.0,22.45,104433.0,25.37,695.0,699.0,0.0,False,False,False,...,False,True,False,False,False,False,False,False,False,True
5,11950.0,13.44,34000.0,10.20,690.0,694.0,0.0,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# X = features, y = target
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale numeric data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=5, batch_size=256, validation_split=0.1)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
3784/3784 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.7907 - loss: 0.4730 - val_accuracy: 0.7994 - val_loss: 0.4596
Epoch 2/5
3784/3784 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8015 - loss: 0.4571 - val_accuracy: 0.7995 - val_loss: 0.4588
Epoch 3/5
3784/3784 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8025 - loss: 0.4562 - val_accuracy: 0.8002 - val_loss: 0.4589
Epoch 4/5
3784/3784 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8006 - loss: 0.4578 - val_accuracy: 0.7997 - val_loss: 0.4586
Epoch 5/5
3784/3784 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8022 - loss: 0.4564 - val_accuracy: 0.7998 - val_loss: 0.4587


In [13]:
from sklearn.metrics import roc_auc_score, f1_score

# Predictions
y_pred_prob = model.predict(X_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

# Metrics
auc = roc_auc_score(y_test, y_pred_prob)
f1 = f1_score(y_test, y_pred)

auc, f1


8409/8409 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step


(np.float64(0.7044465283591836), 0.0923472008217771)

In [14]:
rl_df = df.copy()


In [15]:
def compute_reward(row, action):
    # action: 0 = deny, 1 = approve

    if action == 0:
        return 0

    # approve
    if row['loan_status'] == 0:  # fully paid
        return row['loan_amnt'] * row['int_rate'] / 100
    else:  # default
        return -row['loan_amnt']


In [16]:
# Use DL model predictions as risk scores
import numpy as np

risk_prob = model.predict(X).ravel()

# Action: approve if risk < 0.5
actions = (risk_prob < 0.5).astype(int)

# Compute rewards
rewards = []
for i in range(len(rl_df)):
    r = compute_reward(rl_df.iloc[i], actions[i])
    rewards.append(r)

avg_reward = np.mean(rewards)
avg_reward


42041/42041 ━━━━━━━━━━━━━━━━━━━━ 45s 1ms/step


np.float64(-1644.2319665281607)